<a href="https://colab.research.google.com/github/LoreJob/DeepFake-Dct/blob/main/CNN_for_DeepFake_Detection_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN for DeepFake Detection


# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Ram/Disk on the upper-left part of the colab → Additional Connection Options
- select GPU T4 from the Hardware Accelerator drop-down (You have just 1 hour of using)

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
7.214328854999998
GPU (s):
0.10292108700001279
GPU speedup over CPU: 70x


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## Packages
As the title says, we are using the tensorflow package.

In [4]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

## Importing the dataset




In [5]:
# Height and width of which video frame (you can also do resizing)
HEIGHT, WIDTH = 256, 256

# Number of frames of a video that will be fed to the model as one sequence
N_FRAMES = 20

# Dataset dir
DATABASE_DIR = "Video Dataset Small"

# Classes
CLASSES = ["Real", "Fake"]

### Creating the frame extractor

In [6]:
def frames_extraction(video_path):
    frames_list = []

    #Reading the video
    video_reader = cv2.VideoCapture(Dataset_dir)

    #Getting total number of frames
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    #Selecting only the wanted amount of frames
    skip_frames_window = max(int(video_frames_count/N_FRAMES), 1)

    #Looping to the selected frames
    for frame_counter in range(N_FRAMES):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        
        #reading the frames
        success, frame = video_reader.read()
        #breaking if the read is not successfull
        if not success:
            break
        
        #setting the shape and normalizing the frame (each frame should have a value between 0-1)
        resized_frame = cv2.resize(frame, (HEIGHT, WIDTH))
        normalized_frame = resized_frame / 255

        frames_list.append(normalized_frame)

    video_reader.release()
    return frames_list


### Creating the dataset creator

In [7]:
# Loading videos from folder
def load_videos_from_folder(folder, label):
    videos = []
    labels = []
    for filename in os.listdir(folder):
        video_path = os.path.join(folder, filename)
        frames = frames_extraction(video_path)
        if len(frames) == N_FRAMES:
            videos.append(frames)
            labels.append(label)
    return videos, labels

# Creating the dataset
def create_tf_dataset(video_list, label_list):
    video_tensors = [tf.convert_to_tensor(video, dtype=tf.float32) for video in video_list]
    label_tensors = [tf.convert_to_tensor(label, dtype=tf.int32) for label in label_list]
    dataset = tf.data.Dataset.from_tensor_slices((video_tensors, label_tensors))
    return dataset

### Loading Datasets

In [ ]:
# Define paths
fake_train_dir = os.path.join(DATABASE_DIR, 'fake', 'train')
fake_test_dir = os.path.join(DATABASE_DIR, 'fake', 'test')
fake_val_dir = os.path.join(DATABASE_DIR, 'fake', 'validation')
real_train_dir = os.path.join(DATABASE_DIR, 'real', 'train')
real_test_dir = os.path.join(DATABASE_DIR, 'real', 'test')
real_val_dir = os.path.join(DATABASE_DIR, 'real', 'validation')

# Load videos
fake_train_videos, fake_train_labels = load_videos_from_folder(fake_train_dir, label=1)
fake_test_videos, fake_test_labels = load_videos_from_folder(fake_test_dir, label=1)
fake_val_videos, fake_val_labels = load_videos_from_folder(fake_val_dir, label=1)
real_train_videos, real_train_labels = load_videos_from_folder(real_train_dir, label=0)
real_test_videos, real_test_labels = load_videos_from_folder(real_test_dir, label=0)
real_val_videos, real_val_labels = load_videos_from_folder(real_val_dir, label=0)